# 主动学习智能策略（Agent + LLM）使用指南

本笔记介绍 totol 目录中实现的两种方案：
- 方案1：多臂老虎机（Exp3风格）权重更新 + 融合排序（StrategyAgent）。
- 方案2：LLM/规则决策器（LLMStrategyDecider），默认规则回退。

并演示实际的文件读取与保存位置：
- 数据读取：/home/v-wenliao/gnot/GNOT/data/al_bz 下的 al_labeled.pkl、al_unlabeled.pkl、al_test.pkl
- 临时评估：/home/v-wenliao/gnot/GNOT/data/al_bz/data 下的临时 *.pkl（由 StrategyAgent 在奖励评估时使用）
- 策略权重持久化：/home/v-wenliao/gnot/GNOT/totol/state.json

In [ ]:
import os, sys, json, pickle, time, numpy as np
ROOT = '/home/v-wenliao/gnot/GNOT'
if ROOT not in sys.path: sys.path.append(ROOT)

DATA_DIR = os.path.join(ROOT, 'data', 'al_bz')
STATE_PATH = os.path.join(ROOT, 'totol', 'state.json')
TEMP_DIR = os.path.join(DATA_DIR, 'data')  # StrategyAgent 内部也会用到

print('ROOT =', ROOT)
print('DATA_DIR =', DATA_DIR)
print('STATE_PATH =', STATE_PATH)
print('TEMP_DIR =', TEMP_DIR)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)

## 目录结构与关键文件
- totol/agent_selector.py：方案1，权重更新+融合排序的智能策略选择器。
- totol/strategy_registry.py：注册/封装现有策略，提供回退策略。
- totol/llm_selector.py：方案2，LLM/规则决策器。
- totol/local_strategies.py：随机/幅值/多样性回退策略。
- totol/mock_env.py：合成数据与轻量模型，便于快速演示。
- totol/run_agent_al.py：命令行演示脚本（bandit/llm/hybrid）。
- totol/state.json：跨轮的策略权重持久化文件。

In [ ]:
from totol.mock_env import create_synthetic_al_split
# 若无数据，生成少量合成数据到 DATA_DIR
lp = os.path.join(DATA_DIR, 'al_labeled.pkl')
up = os.path.join(DATA_DIR, 'al_unlabeled.pkl')
tp = os.path.join(DATA_DIR, 'al_test.pkl')
if not (os.path.exists(lp) and os.path.exists(up) and os.path.exists(tp)):
    create_synthetic_al_split(DATA_DIR, n_labeled=8, n_unlabeled=60, n_test=12)

def pkl_info(p):
    sz = os.path.getsize(p) if os.path.exists(p) else 0
    return {'path': p, 'exists': os.path.exists(p), 'size_bytes': sz}

print('Data files:')
print(pkl_info(lp))
print(pkl_info(up))
print(pkl_info(tp))

## 方案1：Agent（多臂老虎机 + 融合排序）
- 每轮仅训练一次模型（如 albz.train_model，示例中用 mock 快速演示）。
- 对每个策略在小候选集上估计“奖励”（误差越大→潜在收益越高）。
- 用 Exp3 累乘更新策略权重。
- 将各策略 Top-K 名单按权重做名次分融合，得全局排序。

In [ ]:
from totol.strategy_registry import build_default_strategies
from totol.agent_selector import StrategyAgent, ALBZ, TEMP_DIR as AGENT_TEMP
from totol.mock_env import build_mock_model_tuple

# 加载数据
import pickle
with open(lp, 'rb') as f: labeled = pickle.load(f)
with open(up, 'rb') as f: unlabeled = pickle.load(f)
with open(tp, 'rb') as f: test = pickle.load(f)
print(f'labeled={len(labeled)}, unlabeled={len(unlabeled)}, test={len(test)}')

# 为了快速演示，使用轻量 mock 模型（若需真实训练可切换为 albz.train_model）
model_tuple = build_mock_model_tuple(labeled[0])

# 构建策略集合（可只保留本地回退策略以保证轻量运行）
all_specs = build_default_strategies()
FAST_DEMO = True
if FAST_DEMO:
    specs = {k: v for k, v in all_specs.items() if k in ('random','magnitude','diversity')}
else:
    specs = all_specs

print('Discovered strategies:', list(specs.keys()))

# 初始化智能选择器
agent = StrategyAgent(strategies=specs, candidate_size=8)
print('Weights before:', json.dumps(agent.weights, indent=2))

# 选择样本索引
selected = agent.select_indices(model_tuple, labeled, unlabeled, select_num=20)
print('Selected indices (Agent):', selected[:20])
print('Weights after:', json.dumps(agent.weights, indent=2))

print('State saved to:', STATE_PATH, 'exists?', os.path.exists(STATE_PATH))
print('Agent TEMP_DIR:', AGENT_TEMP)

## 查看持久化的权重文件 state.json（保存位置演示）

In [ ]:
if os.path.exists(STATE_PATH):
    with open(STATE_PATH, 'r') as f: print(f.read())
else:
    print('state.json not found at', STATE_PATH)

## 方案2：LLM/规则决策器（可插拔）
- 将最近轮的指标（例如 metrics.csv 聚合）输入决策器。
- 未配置 LLM API 时，使用规则回退选择策略。
- 可与方案1结合（Hybrid），作为首选策略来源。

In [ ]:
from totol.llm_selector import LLMStrategyDecider
from totol.strategy_registry import StrategySpec

decider = LLMStrategyDecider()
recent_metrics = {
    'pressure':   [0.12, 0.10],
    'wall-shear': [0.02, 0.02]
}
cand = list(specs.keys())
choice = decider.decide(recent_metrics, cand)
print('LLM/规则选中的策略:', choice)

# 直接调用该策略（统一签名）
spec = specs[choice]
sel_llm = spec.func(model_tuple, labeled, unlabeled, select_num=20)
print('Selected indices (LLM choice):', sel_llm[:20])

## Hybrid示例：LLM首选 + Agent融合
- 先用 LLM 决策一个首选策略，
- 再与 Agent 融合结果，形成最终选择。

In [ ]:
half = 10
base_sel = sel_llm[:max(half, 16)] if sel_llm else []
agent_sel = agent.select_indices(model_tuple, labeled, unlabeled, select_num=20)
# 合并去重
hybrid_sel = list(dict.fromkeys(base_sel[:half] + agent_sel))[:20]
print('Selected indices (Hybrid):', hybrid_sel)

## 小结
- 数据读取：
  - `al_labeled.pkl`, `al_unlabeled.pkl`, `al_test.pkl` 来自 `data/al_bz/`。
- 临时保存：
  - 奖励评估时，`totol/agent_selector.py` 会在 `data/al_bz/data/` 下写入/清理临时 `*.pkl`。
- 权重保存：
  - `totol/state.json` 存储跨轮的策略权重。
- 两种方案可独立使用，也可 **Hybrid** 组合。